In [201]:
%load_ext autoreload
%autoreload 2

import kwant
import numpy as np
import matplotlib.pyplot as plt 
import kwant.continuum
import scipy.sparse.linalg as sla
import tinyarray as tiny
import matplotlib as mpl
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning) 

%matplotlib widget
mpl.rcParams.update({'font.size': 20, 'text.usetex': True,  'font.family': 'serif', 'font.serif': ['Computer Modern']})

sx = tiny.array([[0 , 1] , [1 , 0]])
sy = tiny.array([[0 , -1j] , [1j , 0]])
sz = tiny.array([[1 , 0] , [0 , -1]])


I = tiny.array([[1 , 0] , [0 , 1]])
t_plus = 0.5*(np.kron(sx , I) + 1j*np.kron(sy , I))
t_minus = t_plus.T
phs = np.kron(sy , sy)

The recursive Green's function method relies on these two equations:
$$ \check{G}^{L}_{j} = \left[ i\omega_{p} \check{1} - \check{H}^{0}_{j} - \check{T}^{+}_{j}\check{G}^{L}_{j-1} \check{T}^{-}_{j} \right]^{-1} $$
starting at $j = j_{L}$, which is some slice in the left lead,
$$ \check{G}^{R}_{j} = \left[ i\omega_{p} \check{1} - \check{H}^{0}_{j} - \check{T}^{-}_{j+1}\check{G}^{R}_{j+1} \check{T}^{+}_{j + 1} \right]^{-1} $$
starting at $j = j_{R}$ which is some slice in the right lead,
and finally:
$$\check{G}_{\omega_{p}}(j , j) = \left[ i\omega_{p} \check{1} -  \check{H}^{0}_{j} - \check{T}^{-}_{j+1}\check{G}^{R}_{j+1} \check{T}^{+}_{j + 1}  - \check{T}^{+}_{j}\check{G}^{L}_{j-1} \check{T}^{-}_{j} \right] $$
An upside down hat means its a $(2W \times 2W)$ matrix. $\check{T}^{+}_{j}$ is the matrix that shifts you to the right from site $j$, whilst $\check{T}^{-}_{j}$ shifts you to the left from site $j$. Also $\check{G}^{L/R}_{j} = \check{G}_{\omega_{P}}(j , j)$ which is the Green's function of the system in which all alttice site $j'\geq j+1$ and $j' \leq j - 1$ are deleted.
These means we will first have to evaluate $\check{G}^{L}_{j_{L}-1}$ and $\check{G}^{R}_{j_{R} + 1}$, which this paper tells me how to do numerically: https://journals.aps.org/prb/pdf/10.1103/PhysRevB.44.8017. Probably best to evaluate this numerically since these Green's functions are only analytically soluble in the simplest of cases (i.e. s-wave disorderless superconductors).


From the Ando paper I learned that I should first calculate all modes in the ideal superconducting leads. This should give me a matrix of eigenvectors $U(\pm)$ (where $+$ corresponds to right moving and $-$ corresponds to left-moving) and a diagonal matrix of eigenvalues $\Lambda(\pm)$. We can determine whether they are rightr movers or left movers by studying $arg(\Lambda_{ii})$. The important matrix to calculate is:
$$ F(-) = U(-)\Lambda(-)U^{-1}(-)$$ 

To compute all the modes in the lead, I need to solve the following eigenvalue problem:
$$ \underbrace{\begin{pmatrix} - [\check{T}^{-}]^{-1}(\check{H}^{j}_0 - E) & -[\check{T}^{-}]^{-1}\check{T}^{+} \\ \check{1} & \check{0} \end{pmatrix}}_{\mathbb{M}}\begin{pmatrix}\vec{c}_{j} \\ \vec{c}_{j-1} \end{pmatrix} = \lambda \begin{pmatrix}\vec{c}_{j} \\ \vec{c}_{j-1} \end{pmatrix} $$
where $\check{H}^{j}_{0}$ is the onsite Hamiltonian of slice $j$ and $\vec{c}_{j}$ is the vector of amplitudes on slice $j$. The argument of $\lambda$ will give the direction of current between slice $j -1$ and slice $j$.

In [208]:
from dataclasses import dataclass
from RGFtools import *

@dataclass
class JosephsonParameters:
    W: float
    L: float
    t: float
    mu: float
    Delta: float
    phase: float
    flux: float
    T: float
    n: int

params = JosephsonParameters(W=10, L=10, t=1, mu=0.5, Delta=2, phase=0.0*2*np.pi, flux=0, T=1, n=10)


In [209]:
E = 1j*np.pi*(2*(params.n) + 1)*params.T

F_pos_0 , F_neg_0 = calculate_transfer_matrices(0 , params)
F_pos_Lplusone , F_neg_Lplusone = calculate_transfer_matrices(params.L+1 , params)

# Initial diagonal Green's function in the left-lead at slice 0:
H_0_tilde = h_0(slice , params) - T(slice , -1 , params)@np.linalg.inv(F_neg_0)
G_0 = np.linalg.inv(E - (H_0_tilde))

H_Lplusone_tilde = h_0(params.L+1 , params) - T(params.L+2 , -1 , params)@F_pos_Lplusone
G_Lplusone = np.linalg.inv(E - (H_Lplusone_tilde))

# RECURSION:          
recursion_length = params.L
left_recursion_gfs = np.zeros(shape = (2*params.W , 2*params.W , params.L))
left_recursion_gfs[: , : , 0] = G_0
for i in range(recursion_length):
    if i > 0:
        left_recursion_gfs[: , : , i] = np.linalg.inv(E - h_0(i , params) - T(i , +1 , params)@ left_recursion_gfs[: , : , i-1] @T(i , -1 , params))

right_recursion_gfs = np.zeros(shape = (2*params.W , 2*params.W , params.L))
right_recursion_gfs[: , : , 0] = G_Lplusone
for i in range(params.L):
    if i > 0:
        right_recursion_gfs[: , : , i] = np.linalg.inv(E - h_0(i-1 , params) - T(i , -1 , params)@ right_recursion_gfs[: , : , i] @T(i , +1 , params))


# Evaluate Green function at coordinate eval_point:
eval_point = 5

G_evalpoint = np.linalg.inv(E - h_0(eval_point , params) - T(eval_point , +1 , params)@ left_recursion_gfs[: , : , eval_point-1] @ T(eval_point , -1  , params) -
                            - T(eval_point + 1 , -1 , params) @ right_recursion_gfs[: , : , eval_point + 1] @ T(eval_point + 1 , +1 , params))

# Off diagonal green's functions:
G_plusx = right_recursion_gfs[: , : , eval_point + 1]@T(eval_point + 1 , +1 , params)@G_evalpoint
G_minusx = G_evalpoint@T(eval_point + 1 , -1 , params)@right_recursion_gfs[: , : , eval_point + 1 ]

# Contribution to current:
I_n = np.trace(G_plusx - G_minusx)
print(I_n)

(-2.2729818027755755e-11+1.8676615809454233e-11j)


In [172]:
# slice = 0
# M00 = np.linalg.inv(T(slice + 1 , -1))@(E -  h_0(slice)) # <- Calculating the Hamiltonian at slice j = 0 , which is in the left lead.
# M01 = -np.linalg.inv(T(slice + 1 , -1))@T(slice , +1)
# M10 = np.identity(2*W)
# M11 = np.zeros(shape = (2*W , 2*W))


# M = np.block([[M00 , M01],[M10 , M11]]) #<- Matrix to diagonalise for propagating modes in lead
# evals , evecs = np.linalg.eig(M)


# pos_prop , neg_prop , pos_evanesce , neg_evanesce , list_of_eigenvalues = sorting_modes(evals , evecs  , tol = 1e-4)


# # First I am going to glue together all modes that evanesce/propagate in the positive x-direction:
# pos_modes = np.hstack((pos_prop , pos_evanesce))
# neg_modes = np.hstack((neg_prop , neg_evanesce))

# # The U(\pm)-matrices consisten of amplitudes on the j = 0 slice. So we take only the first half rows:
# U_pos = pos_modes[0:int(pos_modes.shape[0]/2) , :]
# U_neg = neg_modes[0:int(neg_modes.shape[0]/2) , :]

# # The \Lambda(\pm) matrix comprises of all the corresponding eigenvalues:
# Lambda_pos = np.diag(np.hstack((list_of_eigenvalues[0] ,list_of_eigenvalues[2])))
# Lambda_neg = np.diag(np.hstack((list_of_eigenvalues[1] , list_of_eigenvalues[3])))

# # Construct the F(\pm) transfer matrices:
# F_pos = U_pos @ Lambda_pos @ np.linalg.inv(U_pos)
# F_neg = U_neg @ Lambda_neg @ np.linalg.inv(U_neg)


    
# def T(matrix_element):
#     '''
#     Constructs hopping Hamiltonian between neghbouring slices (this is hopping in the x-direction).

#     Inputs:
#     matrix_element: tuple of slice indices (j , J) to construct the hopping that goes from site J -> j
#     pm: Signs of Peierl phases.

#     Returns:
#     T_jJ : ndarray, hopping matrix Hamiltonian.
#     '''
#     j , J = matrix_element[0] , matrix_element[1]

#     # Calculate if j < J in the lattice:
#     sign= np.sign(j - J)
#     index = int(0.5*(1 + sign)) #<- If j > J, then the all the conditionals below must depend on the position of the J coordinate.
#     if matrix_element[index] <= -1 or  matrix_element[index] >= (L+1):
#         return np.kron(np.identity(W) , -t*sz)
    
#     if matrix_element[index]>=0 and matrix_element[index] <= L:
#         peierls_e = -sign*(B/2)*np.arange(W) # array of peierl phases for electrons
#         peierls_h = -peierls_e # array of peierl phases for holes.
#         peierls_interweaved = np.exp(1j*np.vstack((peierls_e , peierls_h)).reshape((-1 , ) , order = 'F')) #<- talking elementwise exponential.
#         peierls_bdg = np.diag(peierls_interweaved) # Peierls phases as the appear in the BdG matrix
#         return np.kron(np.identity(W) , -t*sz)@peierls_bdg